In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [9]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

mbb_teams = {"Miami Hurricanes": ["https://www.sports-reference.com/cbb/schools/miami-fl/men/2023-gamelogs.html"],
             "UConn Huskies": ["https://www.sports-reference.com/cbb/schools/connecticut/men/2023-gamelogs.html"],
             "San Diego State Aztecs": ["https://www.sports-reference.com/cbb/schools/san-diego-state/men/2023-gamelogs.html"],
             "Florida Atlantic Owls": ["https://www.sports-reference.com/cbb/schools/florida-atlantic/men/2023-gamelogs.html"],
             "Gonzaga Bulldogs" : ["https://www.sports-reference.com/cbb/schools/gonzaga/men/2023-gamelogs.html"],
             "UCLA Bruins" : ["https://www.sports-reference.com/cbb/schools/ucla/men/2023-gamelogs.html"],
             "Arkansas Razorbacks" : ["https://www.sports-reference.com/cbb/schools/arkansas/men/2023-gamelogs.html"],
             "Texas Longhorns" : ["https://www.sports-reference.com/cbb/schools/texas/men/2023-gamelogs.html"],
             "Xavier Musketeers" : ["https://www.sports-reference.com/cbb/schools/xavier/men/2023-gamelogs.html"],
             "Houston Cougars" : ["https://www.sports-reference.com/cbb/schools/houston/men/2023-gamelogs.html"],
             "Michigan State Spartans" : ["https://www.sports-reference.com/cbb/schools/michigan-state/men/2023-gamelogs.html"],
             "Kansas State Wildcats" : ["https://www.sports-reference.com/cbb/schools/kansas-state/men/2023-gamelogs.html"],
             "Tennessee Volunteers" : ["https://www.sports-reference.com/cbb/schools/tennessee/men/2023-gamelogs.html"],
             "Princeton Tigers" : ["https://www.sports-reference.com/cbb/schools/princeton/men/2023-gamelogs.html"],
             "Creighton Bluejays" : ["https://www.sports-reference.com/cbb/schools/creighton/men/2023-gamelogs.html"],
             "Alabama Crimson Tide" : ["https://www.sports-reference.com/cbb/schools/alabama/men/2023-gamelogs.html"]}
wbb_teams = {"LSU Tigers": ["https://www.sports-reference.com/cbb/schools/louisiana-state/women/2023-gamelogs.html"],
             "Iowa Hawkeyes": ["https://www.sports-reference.com/cbb/schools/iowa/women/2023-gamelogs.html"],
             "South Carolina Gamecocks" : ['https://www.sports-reference.com/cbb/schools/south-carolina/women/2023-gamelogs.html'],
             "Maryland Terrapins" : ['https://www.sports-reference.com/cbb/schools/maryland/women/2023-gamelogs.html'],
             "Louisville Cardinals" : ['https://www.sports-reference.com/cbb/schools/louisville/women/2023-gamelogs.html'],
             "Miami Hurricanes" : ['https://www.sports-reference.com/cbb/schools/miami-fl/women/2023-gamelogs.html'],
             "Virginia Tech Hokies" : ['https://www.sports-reference.com/cbb/schools/virginia-tech/women/2023-gamelogs.html'],
             "UCLA Bruins" : ['https://www.sports-reference.com/cbb/schools/ucla/women/2023-gamelogs.html'],
             "Ohio State Buckeyes" : ['https://www.sports-reference.com/cbb/schools/ohio-state/women/2023-gamelogs.html'],
             "Notre Dame Fighting Irish" : ['https://www.sports-reference.com/cbb/schools/notre-dame/women/2023-gamelogs.html'],
             "Ole Miss Rebels" : ['https://www.sports-reference.com/cbb/schools/mississippi/women/2023-gamelogs.html'],
             "Colorado Buffaloes" : ['https://www.sports-reference.com/cbb/schools/colorado/women/2023-gamelogs.html'],
             "Villanova Wildcats" : ['https://www.sports-reference.com/cbb/schools/villanova/women/2023-gamelogs.html'],
             "Utah Utes" : ['https://www.sports-reference.com/cbb/schools/utah/women/2023-gamelogs.html'],
             "UConn Huskies" : ['https://www.sports-reference.com/cbb/schools/connecticut/women/2023-gamelogs.html'],
             "Tennessee Volunteers" : ['https://www.sports-reference.com/cbb/schools/tennessee/women/2023-gamelogs.html']}

mbb_soups = []
wbb_soups = []

for team in mbb_teams:
    driver.get(mbb_teams[team][0])
    mbb_teams[team].append( BeautifulSoup(driver.page_source, 'html.parser') )
for team in wbb_teams:
    driver.get(wbb_teams[team][0])
    wbb_teams[team].append( BeautifulSoup(driver.page_source, 'html.parser') )
    
driver.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/111.0.5563.64/chromedriver] found in cache


In [10]:
def soup_to_df(soup, school):
    table = soup.find("table", {"class" : "stats_table"})

    headers = [th.text for th in table.find("thead").find_all("tr")[-1].find_all("th")]
    headers = headers[0:len(headers)-16] + ["OPP " + h for h in headers[len(headers)-16:]]
    headers = headers[0:3] + ['Opponent'] + headers[4:len(headers)]

    rows = [i for i in table.find("tbody").find_all("tr")]
    values = [[r.find("th").text] + [td.text for td in r.find_all("td")] for r in rows]

    df = pd.DataFrame(values, columns=headers)

    df = df[df.FGA.apply(lambda i : str(i).isnumeric())] # filter for empty rows

    cols = ['Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'OPP FG', 'OPP FGA', 'OPP FG%', 'OPP 3P', 'OPP 3PA', 'OPP 3P%', 'OPP FT', 'OPP FTA', 'OPP FT%', 'OPP ORB', 'OPP TRB', 'OPP AST', 'OPP STL', 'OPP BLK', 'OPP TOV', 'OPP PF']
    for c in cols:
        df[c] = pd.to_numeric(df[c])

    df.loc["Total"] = df.sum()

    df = df[['Opponent', 'W/L', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'OPP FG', 'OPP FGA', 'OPP FG%', 'OPP 3P', 'OPP 3PA', 'OPP 3P%', 'OPP FT', 'OPP FTA', 'OPP FT%', 'OPP ORB', 'OPP TRB', 'OPP AST', 'OPP STL', 'OPP BLK', 'OPP TOV', 'OPP PF']]

    df.loc[df.index == "Total", "W/L"] = ""
    df.loc[df.index == "Total", "Opponent"] = "Total"
    
    df['TEAM'] = school
    df['GP'] = len(df) - 1
    df['OREB'] = df.ORB
    df['OPP OREB'] = df['OPP ORB']
    df['REB'] = df.TRB
    df['OPP REB'] = df['OPP TRB']
    df['POSS'] = 0.96 * (df.FGA + df.TOV + 0.44*df.FTA - df.OREB)
    df['OPP POSS'] = 0.96 * (df['OPP FGA'] + df['OPP TOV'] + 0.44*df['OPP FTA'] - df['OPP ORB'])
    df['PTS'] = df.Tm
    df['OPP PTS'] = df.Opp
    df['PPP'] = df.PTS / df['POSS']
    df['OPP PPP'] = df['OPP PTS'] / df['OPP POSS']
    df['FTA%'] = 100 * df.FTA / df.FGA
    df['OPP FTA%'] = 100 * df['OPP FTA'] / df['OPP FGA']
    df['DREB'] = df.TRB - df.ORB
    df['OPP DREB'] = df['OPP TRB'] - df['OPP ORB']
    df['ORB%'] = 100 * df.OREB / (df.OREB + df['OPP DREB'])
    df['OPP ORB%'] = 100 * df['OPP OREB'] / (df['OPP OREB'] + df.DREB)
    df['DRB%'] = 100 * df.DREB / (df.DREB + df['OPP OREB'])
    df['OPP DRB%'] = 100 * df['OPP DREB'] / (df['OPP DREB'] + df.OREB)
    df['REB%'] = 100 * df.REB / (df.REB + df['OPP REB'])
    df['OPP REB%'] = 100 * df['OPP REB'] / (df['OPP REB'] + df.REB)
    df['TO%'] = 100 * df.TOV / df['POSS']
    df['OPP TO%'] = 100 * df['OPP TOV'] / df['OPP POSS']

    return df

In [11]:
logos = pd.read_csv("ncaa_logos.csv")

mbb_games = pd.concat([soup_to_df(mbb_teams[team][1], team) for team in mbb_teams])
mbb_games = pd.merge(mbb_games, logos, on="TEAM", how="left")

wbb_games = pd.concat([soup_to_df(wbb_teams[team][1], team) for team in wbb_teams])
wbb_games = pd.merge(wbb_games, logos, on="TEAM", how="left")

In [12]:
mbb_finalFour = ["UConn Huskies", "Miami Hurricanes", "San Diego State Aztecs", "Florida Atlantic Owls"]
wbb_finalFour = ["Iowa Hawkeyes", "South Carolina Gamecocks",  "Virginia Tech Hokies", "LSU Tigers"]

def isFinalFour(team, women=True):
    if (women and (team in wbb_finalFour)) or ((not women) and (team in mbb_finalFour)):
        return True
    else:
        return False
    
mbb_games['FinalFour'] = mbb_games.TEAM.apply(lambda i : isFinalFour(i, women=False))
wbb_games['FinalFour'] = wbb_games.TEAM.apply(isFinalFour)

In [13]:
mbb_games.to_csv("ncaa_sweet16_mbb_logs.csv")
wbb_games.to_csv("ncaa_sweet16_wbb_logs.csv")